In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from ast import literal_eval
import matplotlib.pyplot as plt

In [7]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer

# Content Recommender

In [8]:
links_small = pd.read_csv('data/input/movies/links_small.csv')
md = pd.read_csv('data/input/movies/movies_metadata.csv')

/var/folders/4v/r4sjb8m14md8bj477_q2b9s00000gn/T/ipykernel_99452/2262967869.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('data/input/movies/movies_metadata.csv')


In [9]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])

**Creating a smaller dataset to work with**

In [10]:
md['id'] = md['id'].astype('int')
smd = md[md['id'].isin(links_small)].copy()
smd.shape

(9099, 24)

In [11]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [12]:
# Analyses 1-2 words in the description of the movies, removing english stop words
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [13]:
tfidf_matrix.shape

(9099, 268124)

**Cosine Similarity**

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows

In [14]:
# Creating a cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
# Resetting the index of the main DataFrame and constructing reverse mapping
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [16]:
def get_recommendations(title):
    """
    Function that returns the top 30 most similar movies to the input title
    """
    # Get the index of the movie title
    idx = indices[title]
    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 30 most similar movies
    sim_scores = sim_scores[1:31]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [17]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

**We are going to use much more suggestive metadata than Overview and Tagline.**

**Build a more sophisticated recommender that takes genre, keywords, cast and crew into consideration.**

In [18]:
credits = pd.read_csv('data/input/movies/credits.csv')
keywords = pd.read_csv('data/input/movies/keywords.csv')

In [19]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [20]:
md.shape

(45463, 24)

In [21]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [22]:
# Consolidates into a single database, filtering for titles that are present in the small database set
smd = md[md['id'].isin(links_small)].copy()
smd.shape

(9219, 27)

**We now have our cast, crew, genres and credits, all in one dataframe**

In [23]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [24]:
def get_director(x):
    """
    Returns the director of the movie
    """

    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [25]:
smd['director'] = smd['crew'].apply(get_director)

In [26]:
# Returns the top 3 actors in the cast
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [27]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [28]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [29]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [30]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s

0                         jealousy
0                              toy
0                              boy
0                       friendship
0                          friends
                   ...            
46427                  psychodrama
46427    husband wife relationship
46427          vacation gone wrong
46427                  ski holiday
46427               swedish family
Length: 64407, dtype: object

In [31]:
s.name = 'keyword'

In [32]:
s = s.value_counts()
s[:5]

keyword
independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: count, dtype: int64

In [33]:
# Removing keywords that appear only once
s = s[s > 1]

In [34]:
# Stemming helps in normalizing text for tasks like text classification
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [35]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [36]:
# Filters for key workds that appear more than once
smd['keywords'] = smd['keywords'].apply(filter_keywords)
# Applies stemming to the keywords
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
# Converts all keywords to lowercase and removes spaces
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [37]:
smd['director'] = smd['director'].apply(lambda x: [x])

In [38]:
# Handles missing values in the genre column and extracting the genres
smd['genres'] = smd['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
# Takes the genre
s = smd.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genres'
smd = smd.drop('genres', axis=1).join(s)

In [39]:
smd['genres'] = smd['genres'].apply(lambda x: [x])

In [46]:
# Creates a soup of all the important metadata
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
# Joins all the words in the soup
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

TypeError: sequence item 4: expected str instance, float found